In [1]:
import pandas as pd
import numpy as np
from hiveplot import HivePlot
import matplotlib.pyplot as plt 
%matplotlib inline
import json
from urllib.request import urlopen, quote
import requests,csv
import pandas as pd #导入这些库后边都要用到

dir = "./"
columns = ['std_id','sex','year','colloge','city']
df = pd.read_csv(dir + "nd_student_info.csv",sep='\t',names=columns)
#分组
cities_stat=df.groupby(by=['city'])['city'].agg({"count":np.size}) 
#排序
cities_stat=cities_stat.reset_index().sort_values(by=["count"],ascending=False)  

E:\envs\anaconda3\envs\tfenv\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


In [2]:
len(cities_stat)

893

In [3]:
#经纬度转换参考https://www.jianshu.com/p/773ff5f08a2c

def getlnglat(address):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'
    ak = 'sqGDDvCDEZPSz24bt4b0BpKLnMk1dv6M'
    add = quote(address) #由于本文城市变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode() #将其他编码的字符串解码成unicode
    temp = json.loads(res)  #对json数据进行解析
    return temp

In [ ]:
df_result = pd.DataFrame(columns=('city','lng','lat','count'))
for indexs in cities_stat.index:
    b = cities_stat.loc[indexs].values[0].strip() #将第一列city读取出来并清除不需要字符
    c= cities_stat.loc[indexs].values[1]  #将第二列price读取出来并清除不需要字符
    try:
        lng = getlnglat(b)['result']['location']['lng'] #采用构造的函数来获取经度
        lat = getlnglat(b)['result']['location']['lat'] #获取纬度
        print(lng)
        print(lat)
        dict = {'city':b,'lng':lng,'lat':lat,'count':c}
        print(dict)
        df_result.loc[df_result.shape[0]+1] = dict
    except:  
        continue  
df_result.to_csv("heat_map.csv",index=False,encoding='utf-8')

116.4133836971231
39.910924547299565
{'city': '北京', 'lng': 116.4133836971231, 'lat': 39.910924547299565, 'count': 840}
104.08153351042463
30.655821878416408
{'city': '成都', 'lng': 104.08153351042463, 'lat': 30.655821878416408, 'count': 291}
106.55843415537664
29.568996245338923
{'city': '重庆', 'lng': 106.55843415537664, 'lat': 29.568996245338923, 'count': 261}
102.85244836500482
24.873998150044006
{'city': '昆明', 'lng': 102.85244836500482, 'lat': 24.873998150044006, 'count': 238}
119.16837791142822
36.71265155126753
{'city': '潍坊', 'lng': 119.16837791142822, 'lat': 36.71265155126753, 'count': 214}
117.12639941261048
36.65655420178723
{'city': '济南', 'lng': 117.12639941261048, 'lat': 36.65655420178723, 'count': 212}
117.21081309155257
39.143929903310074
{'city': '天津', 'lng': 117.21081309155257, 'lat': 39.143929903310074, 'count': 211}
125.15537330312809
42.933308420624584
{'city': '西安', 'lng': 125.15537330312809, 'lat': 42.933308420624584, 'count': 193}
87.62443993536046
43.830763204290484
{

In [ ]:
#打开保存的带有经纬度的数据
cities = pd.read_csv(dir + "heat_map.csv",sep=',')

In [4]:
cities.head()

,city,lng,lat,count
0,北京,116.395645,39.929986,840
1,成都,104.067923,30.679943,291
2,重庆,106.530635,29.544606,261
3,昆明,102.714601,25.049153,238
4,潍坊,119.142634,36.716115,214


In [8]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
import seaborn as sns
import folium
import webbrowser
from folium.plugins import HeatMap

In [9]:
#打开保存的带有经纬度的数据
cities = pd.read_csv(dir + "heat_map.csv",sep=',')
num = len(cities)-1
lat = np.array(cities["lat"][0:num])                        # 获取维度之维度值
lon = np.array(cities["lng"][0:num])                        # 获取经度值
pop = np.array(cities["count"][0:num],dtype=float)    # 获取人口数，转化为numpy浮点型
data1 = [[lat[i],lon[i],pop[i]] for i in range(num)]    #将数据制作成[lats,lons,weights]的形式
map_osm = folium.Map(location=[35,110],zoom_start=5)    #绘制Map，开始缩放程度是5倍
HeatMap(data1).add_to(map_osm)  # 将热力图添加到前面建立的map里

file_path = dir + "heatmap.html"
map_osm.save(file_path)     # 保存为html文件

webbrowser.open(file_path)  # 默认浏览器打开

True